In [131]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot

from pandas_profiling import ProfileReport

In [4]:
WORKDIR = os.getcwd()
DATADIR = os.path.join(WORKDIR, '../data')


In [114]:
df_weather = pd.read_csv(os.path.join(DATADIR, 'weather.csv'), parse_dates = ['data_index_'], index_col = 'data_index_')
df_load = pd.read_csv(os.path.join(DATADIR, 'load_actuals.csv'), parse_dates = ['data_index_'], index_col = 'data_index_')
df_wind = pd.read_csv(os.path.join(DATADIR, 'wind_solar_predictions.csv'), parse_dates = ['data_index_'], index_col = 'data_index_')

In [115]:
#df_weather.info()
#df_load.info()
df_wind.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 50497 entries, 2020-01-01 00:00:00+00:00 to 2021-06-10 00:00:00+00:00
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   solar_prediction_mw  50497 non-null  float64
 1   wind_prediction_mw   50497 non-null  float64
dtypes: float64(2)
memory usage: 1.2 MB


In [116]:
min1 = df_weather.index.min()

In [117]:
max1 = df_weather.index.max()

In [118]:
date = pd.date_range(min1, max1, freq='15T')

In [119]:
date.shape

(50497,)

In [120]:
date = pd.DataFrame(index = date)

In [121]:
date.head()

""
2020-01-01 00:00:00+00:00
2020-01-01 00:15:00+00:00
2020-01-01 00:30:00+00:00
2020-01-01 00:45:00+00:00
2020-01-01 01:00:00+00:00


In [122]:
df = pd.merge(df_wind, date, how = 'right', right_index = True, left_index = True)

In [123]:
df.head()

,solar_prediction_mw,wind_prediction_mw
2020-01-01 00:00:00+00:00,0.0,70.865426
2020-01-01 00:15:00+00:00,0.0,69.296785
2020-01-01 00:30:00+00:00,0.0,66.977409
2020-01-01 00:45:00+00:00,0.0,64.305715
2020-01-01 01:00:00+00:00,0.0,61.128262


In [124]:
df = pd.merge(df_weather, df, how = 'right', right_index = True, left_index = True)

In [125]:
df = pd.merge(df_load, df, how = 'right', right_index = True, left_index = True)

In [126]:
df.head()

,load_actuals_mw,temperature,solar_ghi,solar_prediction_mw,wind_prediction_mw
2020-01-01 00:00:00+00:00,95.756328,274.989655,0.0,0.0,70.865426
2020-01-01 00:15:00+00:00,94.836196,274.925659,0.0,0.0,69.296785
2020-01-01 00:30:00+00:00,93.798127,274.861694,0.0,0.0,66.977409
2020-01-01 00:45:00+00:00,92.162902,274.797699,0.0,0.0,64.305715
2020-01-01 01:00:00+00:00,91.506670,274.423157,0.0,0.0,61.128262


In [127]:
df.to_csv(os.path.join(DATADIR, 'input_data.csv'), index_label='date')

In [139]:
df = pd.read_csv(os.path.join(DATADIR,'input_data.csv'), parse_dates = ['date'])

In [140]:
df.head()

,date,load_actuals_mw,temperature,solar_ghi,solar_prediction_mw,wind_prediction_mw
0,2020-01-01 00:00:00+00:00,95.756328,274.989655,0.0,0.0,70.865426
1,2020-01-01 00:15:00+00:00,94.836196,274.925659,0.0,0.0,69.296785
2,2020-01-01 00:30:00+00:00,93.798127,274.861694,0.0,0.0,66.977409
3,2020-01-01 00:45:00+00:00,92.162902,274.797699,0.0,0.0,64.305715
4,2020-01-01 01:00:00+00:00,91.506670,274.423157,0.0,0.0,61.128262


In [141]:
profile = df.profile_report(
    title="Pandas Profiling Report")

profile.to_file("output.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 111.11it/s]
